# BENCHMARK FUNCTIONS
In questa esercitazione verranno messe a confronto tre tra le librerie di ottimizzazione open source più popolari: **SciPy, PuLP e Pyomo**. Verrà preso in considerazion un singolo caso d'uso che permetterà di evidenziarne le differenze di implementazione e prestazioni.
Si consideri un **problema di trasporto** in cui viene definito un insieme di clienti I = [1,2,3,4,5] e un insieme di fabbriche J = [1,2,3]. Ad ogni cliente corrisponde una domanda di prodotto fissa d_i e ogni fabbrica ha una capacità di produzione fissa M_j. E' necessario considerare i costi di trasporto fissi per consegnare la merce della fabbrica j al cliente i.

Formalmente il problema può essere descritto come segue:

\begin{equation}
\text{Min} \sum_{i \in I}\sum_{j \in J}c_{i,j} x_{i,j} 
\end{equation}

\begin{equation}
\text{subject to} \sum_{j \in J} x_{i,j}=d_{i} \quad  \forall i \in I 
\end{equation}

\begin{equation}
\sum_{i \in I} x_{i,j} <= M_{j} \quad  \forall j \in J 
\end{equation}

\begin{equation}
x_{i,j} >= 0 \quad  \forall i \in I,j \in J 
\end{equation}

L'obiettivo finale è quello fornire la quantità necessaria di merce ad ogni cliente (soddisfacendo la domanda dei clienti e la capacità di produzione delle fabbriche) ad un costo di trasporto totale minimo. Per formulare questo problema dal punto di vista dell'ottimizzazione matematica occorre considerare 3 componenti principali:
- **variabili decisionali**: quantità di merci da inviare dalla fabbrica j al cliente i (numeri reali positivi)
- **vincoli**: la quantità totale di merce deve soddisfare sia la domanda dei clienti che la capacità di produzione della fabbrica
- **funzione obiettivo**: comprensione dei valori delle variabili decisionali tali per cui il costo totale del trasporto sia minimizzato.

## Implementazione
Per prima cosa è necessario definire le strutture dati utili al problema:

In [1]:
import sys
import numpy as np
d = {1:80, 2:270, 3:250, 4:160, 5:180}  # domanda dei clienti
M = {1:500, 2:500, 3:500}               # capacità delle fabbriche
I = [1,2,3,4,5]                         # clienti
J = [1,2,3]                             # fabbriche
cost = {(1,1):4,    (1,2):6,    (1,3):9,
     (2,1):5,    (2,2):4,    (2,3):7,
     (3,1):6,    (3,2):3,    (3,3):3,
     (4,1):8,    (4,2):5,    (4,3):3,
     (5,1):10,   (5,2):8,    (5,3):4
   }                                    # costi di trasporto

Di seguito l'implementazione prosegue con l'utilizzo separato delle tre librerie considerate in questo benchmark:
- SciPy
- Pyomo
- PuLP

### SciPy

In [2]:
# Trasformazione del dizionario dei costi in un array
cost2d = np.empty([len(I), len(J)])
for i in range(len(I)):
    for j in range(len(J)):
        cost2d[i,j] = cost[i+1,j+1]

Definizione delle variabili decisionali

In [3]:
x0 = np.ones(len(cost))*100
bounds = list((0,max(d.values())) for _ in range(cost2d.size))

Definizione della funzione obiettivo

In [4]:
def objective(x):
    obj_func = sum(x[idx]*cost2d[idx//len(J), idx%len(J)] for idx in range(cost2d.size))
    return obj_func

Definizione dei vincoli

In [5]:
# Vincolo: somma delle merci == domanda dei clienti
def const1():
    tmp = []
    for idx in range(0, cost2d.size, len(J)):
        tmp_constr = {
            'type': 'eq',
            'fun': lambda x, idx: d[idx//len(J) + 1] - np.sum(x[idx: idx + len(J)]),
            'args': (idx,)
            }
        tmp.append(tmp_constr)
    return tmp
# Vincolo: somma delle merci <= capacità della fabbrica
def const2():
    tmp = []
    for idx in range(0, cost2d.size, len(I)):
        tmp_constr = {
            'type': 'ineq',
            'fun': lambda x, idx=idx: M[idx//len(I) + 1] - np.sum(x[idx: idx + len(I)])
            }
        tmp.append(tmp_constr)
    return tmp
list_of_lists = [const1(), const2()]
constraints = [item for sublist in list_of_lists for item in sublist]

Calcolo della soluzione del modello tramite il metodo *SLSQP* (Sequential least squares programming).

SLSQP è un metodo iterativo per l'ottimizzazione non lineare vincolata ed è stato scelto per paragonarlo ad approcci più adatti alla programmazione lineare

In [6]:
from scipy.optimize import minimize
solution = minimize(fun = objective,
                x0 = x0,
                bounds = bounds,
                method = 'SLSQP',
                constraints = constraints,
                tol = None,
                callback = None,
                options = {'full_output':False, 'disp':False, 'xtol': 1e-8}
                )

C:\Users\polic\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: OptimizeWarning: Unknown solver options: full_output, xtol
  if __name__ == '__main__':


Verifica dei risultati

In [7]:
if (solution.success) and (solution.status == 0):
    print("La soluzione è feasible e ottimale")
    print("Valore della funzione obiettivo = ", solution.fun)
elif solution.status != 0:
    print("Errore nel ritrovamento della soluzione. ", solution.status)
else:
    print(solution.message)
if solution.success:
    EPS = 1.e-6
    for i,_ in enumerate(solution.x):
        if solution.x[i] > EPS:
            print("Quantità inviata %10s dalla fabbrica %3s al cliente %3s" % (round(solution.x[i]), i%len(J) + 1, i//len(J) + 1))

La soluzione è feasible e ottimale
Valore della funzione obiettivo =  3350.0000000194086
Quantità inviata       80.0 dalla fabbrica   1 al cliente   1
Quantità inviata      270.0 dalla fabbrica   2 al cliente   2
Quantità inviata      125.0 dalla fabbrica   2 al cliente   3
Quantità inviata      125.0 dalla fabbrica   3 al cliente   3
Quantità inviata      160.0 dalla fabbrica   3 al cliente   4
Quantità inviata      180.0 dalla fabbrica   3 al cliente   5


### Pyomo

In [8]:
from pyomo import environ as pe
# ConcreteModel è il modello in cui i dati vengono forniti al momento della definizione. Di contro, AbstractModel necessita di un file di dati
model = pe.ConcreteModel()
# conversione degli iterabili in oggetti
model.d_cust_demand = pe.Set(initialize = d.keys())
model.M_fact_capacity = pe.Set(initialize = M.keys())
# Parametri
model.transport_cost = pe.Param(
    model.d_cust_demand * model.M_fact_capacity,
    initialize = cost,
    within = pe.NonNegativeReals)
model.cust_demand = pe.Param(model.d_cust_demand, 
    initialize = d,
    within = pe.NonNegativeReals)
model.fact_capacity = pe.Param(model.M_fact_capacity, 
    initialize = M,
    within = pe.NonNegativeReals)

Definizione variabili decisionali

In [9]:
model.x = pe.Var(
    model.d_cust_demand * model.M_fact_capacity,
    domain = pe.NonNegativeReals,
    bounds = (0, max(d.values())))

Definizione funzione obiettivo

In [10]:
model.objective = pe.Objective(
    expr = pe.summation(model.transport_cost, model.x),
    sense = pe.minimize)

Definizione dei vincoli

In [11]:
# Vincolo: somma delle merci == domanda dei clienti
def meet_demand(model, customer):
    sum_of_goods_from_factories = sum(model.x[customer,factory] for factory in model.M_fact_capacity)
    customer_demand = model.cust_demand[customer]
    return sum_of_goods_from_factories == customer_demand
model.Constraint1 = pe.Constraint(model.d_cust_demand, rule = meet_demand)
# Vincolo: somma delle merci <= capacità della fabbrica
def meet_capacity(model, factory):
    sum_of_goods_for_customers = sum(model.x[customer,factory] for customer in model.d_cust_demand)
    factory_capacity = model.fact_capacity[factory]
    return sum_of_goods_for_customers <= factory_capacity
model.Constraint2 = pe.Constraint(model.M_fact_capacity, rule = meet_demand)

Calcolo della soluzione.

Tramite la funzione *SolverFactory* è possibile richiamare differenti risolutori sia open source che commerciali, ne sono un esempio *gurobi* o *glpk*. (viene utilizzato gurobi perchè si utilizzerà glpk tramite la libreria successiva)

Il primo non fornisce dettagli sull'implementazione, essendo un software commerciale. 

Il secondo è open source, implementa il metodo del simplesso ed il metodo del punto interno per la risoluzione di problemi lineari. Per la risolzione di problemi interi e misti viene utilizzato il metodo del branch and bound.

In [12]:
solver = pe.SolverFactory("gurobi") #glpk
solution = solver.solve(model)

Verifica dei risultati

In [13]:
from pyomo.opt import SolverStatus, TerminationCondition
if (solution.solver.status == SolverStatus.ok) and (solution.solver.termination_condition == TerminationCondition.optimal):
    print("La soluzione è feasible e ottimale")
    print("Valore della funzione obiettivo = ", model.objective())
elif solution.solver.termination_condition == TerminationCondition.infeasible:
    print ("Errore nel ritrovamento della soluzione. ")
else:
    print(str(solution.solver))
assignments = model.x.get_values().items()
EPS = 1.e-6
for (customer,factory),x in sorted(assignments):
    if x > EPS:
        print("Quantità inviata %10s dalla fabbrica %3s al cliente %3s" % (x, factory, customer))

La soluzione è feasible e ottimale
Valore della funzione obiettivo =  3350.0
Quantità inviata       80.0 dalla fabbrica   1 al cliente   1
Quantità inviata      270.0 dalla fabbrica   2 al cliente   2
Quantità inviata      250.0 dalla fabbrica   3 al cliente   3
Quantità inviata      160.0 dalla fabbrica   3 al cliente   4
Quantità inviata      180.0 dalla fabbrica   3 al cliente   5


## PuLP

Definizione delle variabili decisionali

In [14]:
import pulp
x = pulp.LpVariable.dicts("amount of goods", ((i, j) for i in I for j in J), lowBound = 0, cat = 'Continuous')

Definizione funzione obiettivo

In [15]:
objective = pulp.LpAffineExpression(e = [(x[i,j],cost[i,j]) for i,j in x], name = 'Objective function')
model = pulp.LpProblem(name = "Transportation cost minimization", 
                        sense = pulp.LpMinimize)
model += pulp.lpSum(objective)

C:\Users\polic\Anaconda3\lib\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Definizione dei vincoli

In [16]:
# Vincolo: somma delle merci == domanda dei clienti
for i in I:
    tmpExpression = pulp.LpAffineExpression(e = [(x[i,j], 1) for j in J if (i,j) in x])
    tmpConstraint = pulp.LpConstraint(e = pulp.lpSum(tmpExpression),
        sense = pulp.LpConstraintEQ,                                
        rhs = d[i])
    model.addConstraint(tmpConstraint)
# Vincolo: somma delle merci <= capacità della fabbrica
for j in J:
    tmpExpression = pulp.LpAffineExpression(e = [(x[i,j], 1) for j in J if (i,j) in x])
    tmpConstraint = pulp.LpConstraint(e = pulp.lpSum(tmpExpression),
        sense = pulp.LpConstraintLE,
        rhs = M[j])
    model.addConstraint(tmpConstraint)

Calcolo della soluzione tramite *glpk*

In [17]:
solver = pulp.apis.glpk_api.GLPK_CMD(msg=0)
results = model.solve(solver)

Verifica  dei risultati

In [18]:
if model.status == 1:
    print('La soluzione è: %s' %pulp.LpStatus[model.status])
else:
    print('Errore nel ritrovamento della soluzione: %s' %pulp.LpStatus[model.status])
print('Valore della funzione obiettivo =', pulp.value(model.objective))
EPS = 1.e-6
for (i,j) in x:
    if x[i,j].varValue > EPS:
        print("Quantità inviata %10s dalla fabbrica %3s al cliente %3s" % (x[i,j].varValue,j,i))

La soluzione è: Optimal
Valore della funzione obiettivo = 3350.0
Quantità inviata       80.0 dalla fabbrica   1 al cliente   1
Quantità inviata      270.0 dalla fabbrica   2 al cliente   2
Quantità inviata      250.0 dalla fabbrica   2 al cliente   3
Quantità inviata      160.0 dalla fabbrica   3 al cliente   4
Quantità inviata      180.0 dalla fabbrica   3 al cliente   5


## Discussione dei risultati
Da questa esercitazione si può evincere che tutte e tre le librerie hanno identificato pressocchè lo stesso valore per la funzione obiettivo (anche per via della semplicità del problema).
Tuttavia, il solver *SLSQP* utilizzato in **SciPy** ha ottenuto un valore della funzione obiettivo leggermente peggiore, per via della natura del risolutore più adatto a problemi di programmazione non lineare, rispetto a *GLPK* e *Gurobi* utilizzati da **PuLP** e **Pyomo**.


Le librerie di ottimizzazione prese in esame sono abbastanza differenti sia nella sintassi implementativa che nella filosofia, free o commericale. **SciPy** è probabilmente la più supportata, ha il maggior numero di features e utilizza una semplice sintassi Python. Tuttavia, non supporta i problemi di ottimizzazione binaria.
**PuLP** e **Pyomo** hanno una struttura sintattica piuttosto simile.
**PuLP** è probabilmente la libreria più semplice da utilizzare, tuttavia può gestire solo problemi di ottimizzazione lineare. **Pyomo** risulta avere più seguito di PuLP, supporta problemi di ottimizzazione non lineare e può eseguire anche ottimizzazione multi-obiettivo, sfruttando una grande quantità di risolutori differenti.